# PAIG - Prompt/Reply Guardrails and Observability using OpenAI

This notebook demonstrates how PAIG protects prompts and replies when using OpenAI, as well as how to achieve end-to-end observability.

## Prerequisites

1. **OpenAI API Key**: Required to make API calls to OpenAI.

> The sample prompt is around 12 tokens, and the reply is 15 tokens. The notebook uses the model `gpt-4o-mini`, which currently costs \$0.150 per 1 million input tokens and \$0.600 per 1 million output tokens. Therefore, each prompt/reply costs approximately \$0.00002.

## Details

This notebook covers the following steps:

1. Install Python packages including PAIG Shield Server, PAIG Shield Client, OpenAI, and Spacy models.
2. Start the PAIG Shield Server.
3. Verify that the PAIG Shield Server is up and accepting requests.
4. Generate AI application API key.
5. Set the PAIG AI APP API Key.
6. Configure the OpenAI API Key.
7. Write a simple application Using OpenAI.
8. Write a simple application using OpenAI and the PAIG Shield Client.
9. Test a sample prompt.
10. Test sample prompts with UI.
11. Review access audits in the PAIG Shield Server.
12. Review Application Permissions.
13. Check the reports.

## Exceptions and Assumptions

1. For simplicity, authentication to the PAIG Shield Server is turned off.



# 1. Install Python Packages

This step installs the necessary Python packages for PAIG Shield Server, PAIG Shield Client, OpenAI, and Spacy.

> Note:
> 1. It might take a minute or more to download and install all the packages.
> 2. After everything is installed, you might see a message to restart the runtime. You can ignore this message.


In [ ]:
#!pip install -qqq paig_client openai paig-server --no-warn-conflicts
!pip install -qqq paig_client openai
!python -m spacy download en_core_web_lg
!echo "\n\n"
!echo "Ignore the above messages to restart the runtime or kernel. Please continue to the next step"

# 2. Start the PAIG Shield Server

The command line to start PAIG Shield Server is `paig run`. The server will be started in the background using Python subprocess.

The default port used by PAIG Shield Server is 4545.

> **Tip:** Detailed PAIG application logs can be found in a directory called "logs"

In [ ]:
import subprocess

command = ["paig", "run"]

# Start the PAIG application in the background
# Note - Console logs are hidden using stdout parameter, please remove the stdout parameter to get console logs
process = subprocess.Popen(command, stdout=subprocess.DEVNULL)

print(f"Started PAIG application with PID {process.pid}")


# 3. Verify that the PAIG Shield Server is Up and Accepting Requests

This step ensures that the PAIG Shield Server is running and accepting requests. Once the server is up and running, it will print the URL for the PAIG Shield Server.

> Note: The URL generated will be accessible from outside.  But it may take several seconds for the first load.  The portal will be opened within this Colab notebook in a later step.


In [ ]:
import requests
import time
from google.colab.output import eval_js

url = "http://127.0.0.1:4545/"

print('Please wait while we confirm if your PAIG application is ready...')
while True:
  try:
    response = requests.get(url, timeout=3)
    response.raise_for_status()
    break
  except requests.RequestException:
    print('Server is not ready yet, please hang on...')
    time.sleep(3)

server_url = str(eval_js(f"google.colab.kernel.proxyPort({4545}, {{'cache': true}})"))
print(f'Your PAIG application is now ready!.')

# 4. Generate AI application API key

TThe AI Application API key needs to be exported as "PAIG_APP_API_KEY" to initialize the Privacera Shield library. This API key can be generated from the PAIG portal.

Navigate to Paig Navigator -> **AI Applications**, and select the application for which you want to generate the api key. In the **API KEYS** tab, click the **GENERATE API KEY** button in the top-right corner to generate an API key. Provide a Name and Description, along with a Expiry, or select the Max Validity (1 year) checkbox to set default expiry.

> Note: Once you generate the API key you can view it clicking on eye icon. Make sure to copy and store the key securely.


In [ ]:
from IPython.display import IFrame
ai_application_url = f'{server_url}#/ai_applications'
IFrame(src=ai_application_url, width="100%", height=1000)

# 5. Set the PAIG AI APP API Key
To initialize the PAIG Shield library in your AI application, export the PAIG_APP_API_KEY as an environment variable.

In [ ]:
import os
from getpass import getpass

paig_app_api_key = getpass("🔑 Enter your PAIG AI APP API key and hit Enter:")
os.environ["PAIG_APP_API_KEY"] = paig_app_api_key

# 6. Configure the OpenAI API Key

Enter your OpenAI API key in the text box that will appear when you run this step. After you input the key, press __ENTER__.

> Note: It is important to press __ENTER__ for your value to be accepted.


In [ ]:
import os
from getpass import getpass

openai_api_key = getpass("🔑 Enter your OpenAI API key and hit Enter:")
os.environ["OPENAI_API_KEY"] = openai_api_key
print("OpenAI key has been entered. Now validating it...")

from openai import OpenAI
openai_model = "gpt-4o-mini"
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say Connected to OpenAI successfully!",
        }
    ],
    model=openai_model,
)
print(chat_completion.choices[0].message.content)
print("If connection to OpenAI is successful, then proceed to the next step.")

# 7. Write a Simple Application Using OpenAI

In [ ]:
import openai
import ipywidgets as widgets
from IPython.display import display, HTML

# Set the OPENAI_API_KEY environment variable or set it here
openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Function to generate AI responses
def generate_response(PROMPT):

    try:
        response = openai_client.chat.completions.create(
            model=openai_model,
            messages=[{"role": "user", "content": PROMPT}],
            temperature=0
        )
        llm_response = response.choices[0].message.content
        return llm_response
    except Exception as e:
        return f"Error: {str(e)}"

# UI Components
input_box = widgets.Textarea(
    placeholder="Enter your prompt here...",
    description="Prompt:",
    layout=widgets.Layout(width='100%', height='100px')
)

output_box = widgets.HTML(
    value="<p><i>Your response will appear here...</i></p>",
    placeholder="Generated response",
    description="Output:"
)

generate_button = widgets.Button(description="Generate")

# Callback Function
def on_generate_clicked(b):
    user_prompt = input_box.value
    if user_prompt.strip():
        output_box.value = "<p><i>Generating response...</i></p>"
        response = generate_response(user_prompt)
        output_box.value = f"<p><b>Response:</b><br>{response}</p>"
    else:
        output_box.value = "<p><i>Please enter a valid prompt.</i></p>"

# Link Button to Function
generate_button.on_click(on_generate_clicked)

# Display the UI
display(HTML("<h2>Generative AI App</h2>"))
display(input_box)
display(generate_button)
display(output_box)


# 8. Write a Simple Application Using OpenAI and the PAIG Shield Client

This section demonstrates a simple Python application that uses OpenAI for inference. The PAIG Shield is integrated within the application. The PAIG Shield client is initialized using the `setup()` method and is then used to validate the prompts and replies. In this basic GenAI application, the PAIG Shield's `check_access()` method needs to be explicitly called for the prompt and reply. However, when using frameworks like LangChain, PAIG will automatically instrument the code and call the `check_access()` method for all interactions with LLMs and RAGs.

To enforce user or group-specific policies, the calling username should be set as the request context before processing the prompt. This can be done using the `with paig_shield_client.create_shield_context(username=username):` syntax.

To stitch together related calls, an optional thread ID can be passed to the `check_access()` method to tie them together.

Depending on the policies, the `check_access()` method will perform one of the following actions:

1. If the user is not permitted to use the application or if there is a policy to deny contents which are inappropriate, having unauthorized sensitive information, or is of malicious intent, then the method will throw the exception `paig_client.exception.AccessControlException`. This exception can be caught, and an alternate reply can be returned to the caller.
2. If the request is permitted but contains PII or sensitive information that is not authorized and needs to be redacted, the method will return the content with the PII or sensitive data elements redacted. This behavior is consistent for prompts, requests to RAGs, replies from RAGs, requests to LLMs, and replies from LLMs.
3. If there are no policy violations, the content is returned without any alterations.



In [ ]:

from paig_client import client as paig_shield_client
from paig_client.model import ConversationType
import paig_client.exception
from openai import OpenAI
import uuid

# Set the OPENAI_API_KEY environment variable or set it here
openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# PAIG supports frameworks like LangChain and VectorDBs like Milvus, OpenSearch. The integration to be considered should be passed as the frameworks parameter.
paig_shield_client.setup(frameworks=[])

# Create a function which can be called for the prompts
def query_as_user(username, prompt_text):
    # Generate a random UUID which will be used to bind a prompt with a reply
    privacera_thread_id = str(uuid.uuid4())

    try:
        with paig_shield_client.create_shield_context(username=username):
            print(f"PROMPT BY USER: {prompt_text}")

            # Validate prompt with Privacera Shield
            updated_prompt_text = paig_shield_client.check_access(
                text=prompt_text,
                conversation_type=ConversationType.PROMPT,
                thread_id=privacera_thread_id
            )
            updated_prompt_text = updated_prompt_text[0].response_text
            if prompt_text != updated_prompt_text:
                print(f"Updated Prompt Text: {updated_prompt_text}")
            else:
                print("PROMPT VALIDATION: Prompt has not been changed by PAIG.")

            # Call LLM with updated prompt text
            PROMPT = f"""Use the following pieces of context to answer the question at the end.
            {updated_prompt_text}
            ANSWER:
            """
            response = openai_client.chat.completions.create(
                model=openai_model,
                messages=[{"role": "user", "content": PROMPT}],
                temperature=0
            )
            llm_response = response.choices[0].message.content
            print(f"LLM Response: {llm_response}")

            # Validate LLM response with Privacera Shield
            updated_reply_text = paig_shield_client.check_access(
                text=llm_response,
                conversation_type=ConversationType.REPLY,
                thread_id=privacera_thread_id
            )
            updated_reply_text = updated_reply_text[0].response_text
            if llm_response != updated_reply_text:
                print(f"REPLY VALIDATION (UPDATED BY PAIG): {updated_reply_text}")
            else:
                print("REPLY VALIDATION: The reply has not been updated by PAIG.")
            return updated_reply_text
    except paig_client.exception.AccessControlException as e:
        # If access is denied, this exception will be thrown. Handle it accordingly.
        print("The query has been denied!")
        print(f"AccessControlException: {e}")
        return "DENIED: Prompt is not authorized."
print("PAIG Shield setup successfully completed! You can now proceed to the next step.")



# 9. Test a Sample Prompt

We will call the method `query_as_user` using a test user named `sally` with a sample prompt.

Since we are using the demo application configuration, which has a policy that redacts PERSON_NAME from replies, any elements matching the policy in the LLM's reply will be redacted before responding back to the caller.


In [ ]:

# Using test user named sally
reply = query_as_user("sally", "Who was the first President of USA and where did they live?")
print(f"REPLY TO USER={reply}")

# 10. Test sample prompts with UI

In [ ]:
import openai
import ipywidgets as widgets
from IPython.display import display, HTML
import html


# Function to generate AI responses
def generate_response(PROMPT):

    try:
        llm_response = query_as_user("sally", PROMPT)
        return llm_response
    except Exception as e:
        return f"Error: {str(e)}"

# UI Components
input_box = widgets.Textarea(
    placeholder="Enter your prompt here...",
    description="Prompt:",
    layout=widgets.Layout(width='100%', height='100px')
)

output_box = widgets.HTML(
    value="<p><i>Your response will appear here...</i></p>",
    placeholder="Generated response",
    description="Output:"
)

generate_button = widgets.Button(description="Generate")

# Callback Function
def on_generate_clicked(b):
    user_prompt = input_box.value
    if user_prompt.strip():
        output_box.value = "<p><i>Generating response...</i></p>"
        response = generate_response(user_prompt)
        sanitized_response = html.escape(response)
        output_box.value = f"<p><b>Response:</b><br>{sanitized_response}</p>"
    else:
        output_box.value = "<p><i>Please enter a valid prompt.</i></p>"

# Link Button to Function
generate_button.on_click(on_generate_clicked)

# Display the UI
display(HTML("<h2>Generative AI App</h2>"))
display(input_box)
display(generate_button)
display(output_box)


# 11. Review Access Audits in the PAIG Shield Server

In this step, we will open the PAIG Server portal and check the audits. The portal will be embedded within this notebook.

1. In the PAIG portal, navigate to the `Paig Lens > Access Audits` section. You will see the audit record from the above request.
2. Click on the `More Details` button to see the details of the prompts sent by the application to the LLM and the responses coming from the LLM.
3. PAIG will identify all PII and sensitive data and tag them.
4. The default policy is to redact PERSON_NAME, so the president's name will be redacted before being sent to the caller.


In [ ]:
from IPython.display import IFrame

audit_url = f'{server_url}#/audits_security'
IFrame(src=audit_url, width="100%", height=1000)



# 12. Review Application Permissions

1. In the portal, go to `Paig Navigator -> AI Applications -> PAIG Demo`.
2. Click on the `PERMISSIONS` tab at the top.
3. You will see a policy stating that any reply containing `PERSON`, `EMAIL_ADDRESS`, or `PHONE_NUMBER` should be redacted.



# 13. Check the Reports

1. Click on `Paig Lens -> Reports -> Sensitive Data Access Overview`.
2. This report provides statistics on the PII and other sensitive data found in the prompts and replies.
3. Similarly, the report `Summary of Users who accessed the GenAI Application` will provide details about the GenAI applications and the users accessing them.
